# Data analysis on PC Usage Analyser Data


In [346]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime

from collections import Counter

## Import Data


In [347]:
# get all the csv files from the documents directory, and combine them.
import os
import glob

path = os.path.expanduser("~/Documents/PC Usage Analyzer Data/")
path

'C:\\Users\\Krishnaraj/Documents/PC Usage Analyzer Data/'

In [348]:
all_files = glob.glob(path + "/*.csv")
# ignore all the files that contain the word collaborative in the filename
all_files = [file for file in all_files if "collaborative" not in file]

# if any of their sizes are 0, remove them
all_files = [file for file in all_files if os.stat(file).st_size != 0]

all_files

['C:\\Users\\Krishnaraj/Documents/PC Usage Analyzer Data\\2024-02-13 12-58-13.csv',
 'C:\\Users\\Krishnaraj/Documents/PC Usage Analyzer Data\\2024-02-23 00-44-44.csv',
 'C:\\Users\\Krishnaraj/Documents/PC Usage Analyzer Data\\2024-03-04 17-49-46.csv',
 'C:\\Users\\Krishnaraj/Documents/PC Usage Analyzer Data\\2024-03-10 01-21-35.csv',
 'C:\\Users\\Krishnaraj/Documents/PC Usage Analyzer Data\\2024-03-17 21-30-45.csv',
 'C:\\Users\\Krishnaraj/Documents/PC Usage Analyzer Data\\2024-03-22 21-30-49.csv',
 'C:\\Users\\Krishnaraj/Documents/PC Usage Analyzer Data\\2024-03-29 20-47-20.csv',
 'C:\\Users\\Krishnaraj/Documents/PC Usage Analyzer Data\\2024-04-05 17-51-22.csv',
 'C:\\Users\\Krishnaraj/Documents/PC Usage Analyzer Data\\data.csv']

In [349]:
# combine all the files into one dataframe
# now iterate over all the files and combine them into one dataframe

data = pd.DataFrame()

for filename in all_files:
    current_file_data = pd.read_csv(
        os.path.join(path, filename),
        dtype={
            "Title": str,
            "Start Time": int,
            "Registered End Time": int,
            "Real Duration": str,
        },
    )
    # append to data
    data = pd.concat([data, current_file_data])

In [350]:
data.shape

(173500, 4)

## Pre Process Data


In [351]:
data.head()

# convert start time to datetime
data["Start Time"] = data["Start Time"].apply(
    lambda x: datetime.datetime.fromtimestamp(x).strftime("%d %B %Y %H:%M:%S")
)

data["Registered End Time"] = data["Registered End Time"].apply(
    lambda x: datetime.datetime.fromtimestamp(x).strftime("%d %B %Y %H:%M:%S")
)

data.head()

Title                 Start Time        Registered End Time    Real Duration
0  idle  05 February 2024 18:28:57  05 February 2024 18:29:07  0 days 00:00:10
1  idle  05 February 2024 18:29:07  05 February 2024 18:29:17  0 days 00:00:10
2  idle  05 February 2024 18:29:17  05 February 2024 18:29:27  0 days 00:00:10
3  idle  05 February 2024 18:29:27  05 February 2024 18:29:37  0 days 00:00:10
4  idle  05 February 2024 18:29:37  05 February 2024 18:29:47  0 days 00:00:10

In [352]:
# now sort by start time
data_copy = data.copy()

In [353]:
# list all data types
data.dtypes

Title                  object
Start Time             object
Registered End Time    object
Real Duration          object
dtype: object

In [354]:
# convert all duration to pandas timedelta
data["Real Duration"] = pd.to_timedelta(data["Real Duration"])
# convert all start time to pandas datetime
data["Start Time"] = pd.to_datetime(data["Start Time"])
# convert all end time to pandas datetime
data["Registered End Time"] = pd.to_datetime(data["Registered End Time"])

data.dtypes

Title                           object
Start Time              datetime64[ns]
Registered End Time     datetime64[ns]
Real Duration          timedelta64[ns]
dtype: object

In [355]:
# now sort by start time
data = data.sort_values(by="Start Time")
data.head()

Title          Start Time Registered End Time  \
184    Windows PowerShell 2024-02-05 18:14:10 2024-02-05 19:00:50   
158  To-Do | Default view 2024-02-05 18:16:05 2024-02-05 18:56:19   
149              WhatsApp 2024-02-05 18:17:22 2024-02-05 18:54:17   
148               Desktop 2024-02-05 18:21:17 2024-02-05 18:54:07   
0                    idle 2024-02-05 18:28:57 2024-02-05 18:29:07   

      Real Duration  
184 0 days 00:00:10  
158 0 days 00:00:10  
149 0 days 00:00:10  
148 0 days 00:00:10  
0   0 days 00:00:10

In [356]:
# now remove any null values
data = data.dropna()

In [357]:
data.shape

(173500, 4)

In [358]:
# remove any non ascii characters from the title
data["Title"] = data["Title"].apply(lambda x: "".join(i for i in x if ord(i) < 128))

# let us make all title values lowercase.
data["Title"] = data["Title"].apply(lambda x: x.lower())

# now remove any rows where the title is empty
data = data[data["Title"] != ""]

data.shape

(173500, 4)

In [359]:
# reorder the index to start from 1, make sure they are integers
data.index = np.arange(1, len(data) + 1)
data.head()

Title          Start Time Registered End Time  \
1    windows powershell 2024-02-05 18:14:10 2024-02-05 19:00:50   
2  to-do | default view 2024-02-05 18:16:05 2024-02-05 18:56:19   
3              whatsapp 2024-02-05 18:17:22 2024-02-05 18:54:17   
4               desktop 2024-02-05 18:21:17 2024-02-05 18:54:07   
5                  idle 2024-02-05 18:28:57 2024-02-05 18:29:07   

    Real Duration  
1 0 days 00:00:10  
2 0 days 00:00:10  
3 0 days 00:00:10  
4 0 days 00:00:10  
5 0 days 00:00:10

In [360]:
# anytime idle is written, we know that user was idle for 300 seconds. We can replace this with idle, as the user wasnt really doing that for those 300 seconds. 30 rows because each row is 10 seconds.

# let us find the indices of all rows where idle appears after something not idle.

# this should only be done once.

for index, row in data.iterrows():
    # if we encounter idle, check if previous row was idle.
    if "idle" in row["Title"]:
        # check if previous row was idle
        if "idle" not in data.loc[index - 1, "Title"]:
            # replace the previous 30 rows to idle
            data.loc[index - 30 : index, "Title"] = "idle"

In [361]:
data.head()

Title          Start Time Registered End Time   Real Duration
1  idle 2024-02-05 18:14:10 2024-02-05 19:00:50 0 days 00:00:10
2  idle 2024-02-05 18:16:05 2024-02-05 18:56:19 0 days 00:00:10
3  idle 2024-02-05 18:17:22 2024-02-05 18:54:17 0 days 00:00:10
4  idle 2024-02-05 18:21:17 2024-02-05 18:54:07 0 days 00:00:10
5  idle 2024-02-05 18:28:57 2024-02-05 18:29:07 0 days 00:00:10

In [362]:
data.dtypes

Title                           object
Start Time              datetime64[ns]
Registered End Time     datetime64[ns]
Real Duration          timedelta64[ns]
dtype: object

In [363]:
# data is now ready to be analyzed

# Exploratory Data Analysis


In [110]:
statistics = {}

### Most used applications


In [154]:
no_idle_data = data[data["Title"] != "idle"]
no_idle_data.shape

# add to statistics
statistics["Idle Time (s)"] = (data.shape[0] - no_idle_data.shape[0]) * 10

In [155]:
collab_data = pd.DataFrame(columns=["Title", "Real Duration"])

In [156]:
title_counter = Counter()

# iterate over all the rows
for row in no_idle_data.itertuples():
    title_counter[row.Title] += 10

In [157]:
# add the counter data to collab_data, make sure real duartion is in pd.Timedelta
collab_data = pd.DataFrame(
    {
        "Title": list(title_counter.keys()),
        "Real Duration": list(title_counter.values()),
    }
)

collab_data["Real Duration"] = pd.to_timedelta(collab_data["Real Duration"], unit="s")

In [158]:
collab_data

Title   Real Duration
0                                               desktop 0 days 05:56:40
1                                              whatsapp 1 days 13:13:20
2                                          whatcha doin 0 days 00:02:40
3                                                 games 0 days 01:01:00
4     krishnarajt/attendence-assistant: mini project... 0 days 00:00:20
...                                                 ...             ...
4055  [3][neural net little last part] data science ... 0 days 00:00:10
4056            #minecraft | backchodi bazaar - discord 0 days 00:01:00
4057                                    tlauncher setup 0 days 00:00:30
4058                                          tlauncher 0 days 00:00:10
4059                                    tlauncher 2.919 0 days 00:00:10

[4060 rows x 2 columns]

### Fixing Browser Tabs


In [159]:
# the browser info is kept like "tab title - browser name"
# out of the unique titles we got above, lets find only browser related ones.

browsers_names = [
    "microsoft edge",
    "google chrome",
    "firefox",
    "opera",
    "safari",
    "brave",
    "opera gx",
    "vivaldi",
    "chromium",
    "internet expolorer",
]

browser_data = collab_data[collab_data["Title"].str.contains("|".join(browsers_names))]

# remove all these rows from collab_data
collab_data = collab_data[~collab_data["Title"].str.contains("|".join(browsers_names))]

In [160]:
browser_data

Title   Real Duration
4     krishnarajt/attendence-assistant: mini project... 0 days 00:00:20
5     editing attendence-assistant/readme.md at main... 0 days 00:00:40
9     1337x.to and 19 more pages - personal - micros... 0 days 00:00:20
10    chatgpt and 9 more pages - personal - microsof... 0 days 00:17:30
11    download free clock icons | reshot and 10 more... 0 days 00:00:10
...                                                 ...             ...
4041  (168) statquest with josh starmer - youtube an... 0 days 00:01:00
4046  ty csf a and 2 more pages - personal - microso... 0 days 00:00:10
4049  united states government configuration baselin... 0 days 00:01:00
4050  intern and 4 more pages - personal - microsoft... 0 days 00:00:40
4051  online test for intern and 4 more pages - pers... 0 days 00:01:50

[2396 rows x 2 columns]

In [161]:
# add a new row titled browser that sums up all this to the collab_data
collab_data.loc[len(collab_data)] = ["browser", browser_data["Real Duration"].sum()]

### lets us now look at other commonly clubbed applications


In [241]:
vs_code_data = collab_data[collab_data["Title"].str.contains("visual studio code")]

# remove all these rows from collab_data
collab_data = collab_data[~collab_data["Title"].str.contains("visual studio code")]

In [243]:
vs_code_data

Title   Real Duration
2353  schedule.jsx - appointment-assistant - web dev... 0 days 05:34:40
1631  client_uploads.py - backend - python - visual ... 0 days 03:21:30
2312  sidenav.jsx - appointment-assistant - web dev ... 0 days 03:07:10
1927  login.jsx - appointment-assistant - web dev - ... 0 days 02:54:10
1813  views.py - pc usage analyzer - python - visual... 0 days 02:21:20
...                                                 ...             ...
3145   welcome - python - visual studio code - insiders 0 days 00:00:10
3115  transforming_to_normal.ipynb - course 2 - stat... 0 days 00:00:10
3114  eda on titanic.ipynb - course 2 - statistics a... 0 days 00:00:10
3108  extension: python extension pack - course 2 - ... 0 days 00:00:10
3029  server.js - appointment-assistant - web dev - ... 0 days 00:00:10

[497 rows x 2 columns]

In [244]:
# add a new row titled vs code that sums up all this to the collab_data
collab_data.loc[len(collab_data)] = ["vs code", vs_code_data["Real Duration"].sum()]

C:\Users\Krishnaraj\AppData\Local\Temp\ipykernel_30580\1998736994.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collab_data.loc[len(collab_data)] = ["vs code", vs_code_data["Real Duration"].sum()]


In [245]:
collab_data

Title   Real Duration
1664                                  browser 3 days 02:38:40
1                                    whatsapp 1 days 13:13:20
1916                   sniper ghost warrior 3 0 days 13:35:30
1684                               valorant   0 days 09:12:30
1319                            battlefield v 0 days 06:17:50
...                                       ...             ...
3043                  dvd drive (e:) n24ur37w 0 days 00:00:10
3040                      rufus update policy 0 days 00:00:10
1542  (0%) converting images - directory opus 0 days 00:00:10
4059                          tlauncher 2.919 0 days 00:00:10
1167                                  vs code 3 days 03:41:20

[1168 rows x 2 columns]

In [246]:
# so we are down to almost 1/4th of the data now
collab_data.shape

(1168, 2)

In [248]:
# lets get spotify
spotify_data = collab_data[collab_data["Title"].str.contains("spotify")]

# remove all these rows from collab_data
collab_data = collab_data[~collab_data["Title"].str.contains("spotify")]

spotify_data

Title   Real Duration
1918  spotify - unappreciated beauty  mikolai stroin... 0 days 00:25:30
851   spotify - temp - playlist by krishnaraj pt | s... 0 days 00:12:50
156            spotify - web player: music for everyone 0 days 00:10:20
465   spotify - 1.0 the apology of a bumbling histor... 0 days 00:06:30
853                     spotify - dus bahane  shaan  kk 0 days 00:03:30
3107                                            spotify 0 days 00:01:20
3509  spotify - smooth criminal - 2012 remaster  mic... 0 days 00:01:10
278   spotify - mitwa  shankar-ehsaan-loy  shankar m... 0 days 00:00:50
852               spotify - counting stars  onerepublic 0 days 00:00:50
467   spotify - history of the world podcast | podca... 0 days 00:00:40
466   spotify - vol 1 ep 1 - introduction to the pre... 0 days 00:00:40
1454                       spotify - maina maina  shaan 0 days 00:00:30
553                   spotify - akuma no ko  higuchi ai 0 days 00:00:30
1485                          spotify - sayonee  junoon 0 days 00:00:20
1458         spotify - do u know  shaan  shreya ghoshal 0 days 00:00:20
1456         spotify - this is shaan | spotify playlist 0 days 00:00:20
1455  spotify - my dil goes mmmm  vishal-shekhar  sh... 0 days 00:00:20
1697     spotify - gali mein chand (happy)  alka yagnik 0 days 00:00:20
888       spotify - alone  pt. ii  alan walker  ava max 0 days 00:00:10
4009                                  spotify installer 0 days 00:00:10
4012                                       spotify free 0 days 00:00:10
709   spotify - bolo na (from "12th fail") - single ... 0 days 00:00:10
854    spotify - jab se tere naina  monty sharma  shaan 0 days 00:00:10
1698  spotify - alka yagnik ke nagme - album by alka... 0 days 00:00:10
2544  spotify - macarena - bayside boys remix  los d... 0 days 00:00:10
1290                   spotify - mary on a cross  ghost 0 days 00:00:10
2962  spotify - yeh sham mastani  kishore kumar  r. ... 0 days 00:00:10
1433  spotify - aarti kunj bihari ki  lakhbir singh ... 0 days 00:00:10
1447                                    spotify  search 0 days 00:00:10
1448  spotify - thats all i really wanna do  shaan  ... 0 days 00:00:10
1457  spotify - behti hawa sa tha woh  shantanu moit... 0 days 00:00:10
1463  spotify - koi kahe kehta rahe  shankar mahadev... 0 days 00:00:10
1484                   spotify - cheliya cheliya  shaan 0 days 00:00:10
1486  spotify - o humdum suniyo re (from "saathiya")... 0 days 00:00:10
1487     spotify - phir se sajaa do duniya saari  shaan 0 days 00:00:10
1488  spotify - rimjhim e dharate  shaan  shreya gho... 0 days 00:00:10

In [249]:
# add a new row titled spotify that sums up all this to the collab_data
collab_data.loc[len(collab_data)] = ["spotify", spotify_data["Real Duration"].sum()]

C:\Users\Krishnaraj\AppData\Local\Temp\ipykernel_30580\3148905242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collab_data.loc[len(collab_data)] = ["spotify", spotify_data["Real Duration"].sum()]


In [251]:
collab_data.shape

(1133, 2)

In [256]:
# lets sort collab data
collab_data = collab_data.sort_values(by="Real Duration", ascending=False)
collab_data

# lets separate out the rows which are not top 50
others_collab_data = collab_data[50:]
collab_data = collab_data[:50]

# now lets separate all the rows in others_collab_data which dont have a hyphen
file_manager_data = others_collab_data[others_collab_data["Title"].str.contains("-")]

# remove all these rows from others_collab_data
others_collab_data = others_collab_data[~others_collab_data["Title"].str.contains("-")]

# add a new row titled others in collab_data
collab_data.loc[len(collab_data)] = [
    "others",
    others_collab_data["Real Duration"].sum(),
]

# add a new row titled file manager in collab_data
collab_data.loc[len(collab_data)] = [
    "file manager",
    file_manager_data["Real Duration"].sum(),
]

collab_data

Title   Real Duration
1167                                            vs code 3 days 03:41:20
1664                                            browser 3 days 02:38:40
1                                              whatsapp 1 days 13:13:20
1916                             sniper ghost warrior 3 0 days 13:35:30
1684                                         valorant   0 days 09:12:30
1319                                      battlefield v 0 days 06:17:50
0                                               desktop 0 days 05:56:40
715                         stremio - freedom to stream 0 days 02:15:10
58                 mail - krishnaraj thadesar - outlook 0 days 02:11:50
2141               rocket league (64-bit  dx11  cooked) 0 days 01:59:10
1329                                 forza motorsport 7 0 days 01:53:20
87                                      program manager 0 days 01:50:20
81                                          dying light 0 days 01:47:40
150                              call of duty: blackops 0 days 01:45:10
2931                                    forza horizon 5 0 days 01:41:20
1435                                 backend  readme.md 0 days 01:40:00
2369                           a plague tale: innocence 0 days 01:36:30
7                                    windows powershell 0 days 01:28:20
1963                         devtools - localhost:5173/ 0 days 01:25:30
2360            devtools - localhost:5173/user_schedule 0 days 01:23:40
6                                  to-do | default view 0 days 01:22:10
63                                   stuff worth noting 0 days 01:22:00
193                          need for speed most wanted 0 days 01:21:30
8                                             downloads 0 days 01:16:00
1132                                            spotify 0 days 01:09:50
1723                             the last of us part i  0 days 01:07:30
2140                                 grand theft auto v 0 days 01:03:10
3094                                 f1 22 (directx 12) 0 days 01:01:10
3                                                 games 0 days 01:01:00
684                                             rstudio 0 days 00:54:20
1392                               microsoft whiteboard 0 days 00:53:10
18          p:\programs\dsml\whatcha doin\dist\main.exe 0 days 00:52:00
3737                                          em client 0 days 00:51:10
197                                          gameface   0 days 00:51:00
1732                                             images 0 days 00:50:20
26    call of duty: modern warfare 2 campaign remast... 0 days 00:44:10
1452                                backend  facerec.py 0 days 00:43:40
184                                            expenses 0 days 00:42:20
401                               free download manager 0 days 00:42:20
1323                                           hitman 3 0 days 00:41:30
2234                                  hogwarts legacy   0 days 00:38:50
2398                     sniper ghost warrior contracts 0 days 00:36:40
1445                                   backend  test.py 0 days 00:36:20
22                                             settings 0 days 00:36:10
448                                 notification center 0 days 00:34:40
83            marvel's spider-man remastered v1.812.1.0 0 days 00:34:40
3474  unit-ii-understanding virtualization.pdf - fox... 0 days 00:34:30
198                                             this pc 0 days 00:33:10
2274          devtools - localhost:5173/new_appointment 0 days 00:30:40
3407                                   plots and graphs 0 days 00:28:00
50                                               others 1 days 00:10:00
51                                         file manager 0 days 15:53:40

In [280]:
# now sort this data
collab_data = collab_data.sort_values(by="Real Duration", ascending=False)

In [260]:
# lets rewrite the indices to start from 1
collab_data.index = np.arange(1, len(collab_data) + 1)
collab_data

Title   Real Duration
1                                             vs code 3 days 03:41:20
2                                             browser 3 days 02:38:40
3                                            whatsapp 1 days 13:13:20
4                                              others 1 days 00:10:00
5                                        file manager 0 days 15:53:40
6                              sniper ghost warrior 3 0 days 13:35:30
7                                          valorant   0 days 09:12:30
8                                       battlefield v 0 days 06:17:50
9                                             desktop 0 days 05:56:40
10                        stremio - freedom to stream 0 days 02:15:10
11               mail - krishnaraj thadesar - outlook 0 days 02:11:50
12               rocket league (64-bit  dx11  cooked) 0 days 01:59:10
13                                 forza motorsport 7 0 days 01:53:20
14                                    program manager 0 days 01:50:20
15                                        dying light 0 days 01:47:40
16                             call of duty: blackops 0 days 01:45:10
17                                    forza horizon 5 0 days 01:41:20
18                                 backend  readme.md 0 days 01:40:00
19                           a plague tale: innocence 0 days 01:36:30
20                                 windows powershell 0 days 01:28:20
21                         devtools - localhost:5173/ 0 days 01:25:30
22            devtools - localhost:5173/user_schedule 0 days 01:23:40
23                               to-do | default view 0 days 01:22:10
24                                 stuff worth noting 0 days 01:22:00
25                         need for speed most wanted 0 days 01:21:30
26                                          downloads 0 days 01:16:00
27                                            spotify 0 days 01:09:50
28                             the last of us part i  0 days 01:07:30
29                                 grand theft auto v 0 days 01:03:10
30                                 f1 22 (directx 12) 0 days 01:01:10
31                                              games 0 days 01:01:00
32                                            rstudio 0 days 00:54:20
33                               microsoft whiteboard 0 days 00:53:10
34        p:\programs\dsml\whatcha doin\dist\main.exe 0 days 00:52:00
35                                          em client 0 days 00:51:10
36                                         gameface   0 days 00:51:00
37                                             images 0 days 00:50:20
38  call of duty: modern warfare 2 campaign remast... 0 days 00:44:10
39                                backend  facerec.py 0 days 00:43:40
40                                           expenses 0 days 00:42:20
41                              free download manager 0 days 00:42:20
42                                           hitman 3 0 days 00:41:30
43                                  hogwarts legacy   0 days 00:38:50
44                     sniper ghost warrior contracts 0 days 00:36:40
45                                   backend  test.py 0 days 00:36:20
46                                           settings 0 days 00:36:10
47                                notification center 0 days 00:34:40
48          marvel's spider-man remastered v1.812.1.0 0 days 00:34:40
49  unit-ii-understanding virtualization.pdf - fox... 0 days 00:34:30
50                                            this pc 0 days 00:33:10
51          devtools - localhost:5173/new_appointment 0 days 00:30:40
52                                   plots and graphs 0 days 00:28:00

In [307]:
collab_data.dtypes

# capitalize the first letter of each word in the title
collab_data["Title"] = collab_data["Title"].apply(lambda x: x.title())

In [308]:
def line_plot(
    self, x, y, title, subtitle, xlabel, ylabel, color_scheme, size=(10, 6)
) -> tuple:
    """Creates a line plot with the given data
    and then returns the figure and axis objects.
    """

    # first create the line plot
    fig, ax = plt.subplots(figsize=size)
    ax.plot(x, y, color=self.color_scheme.primary)

    # now formatting

    # axes

    # tilt the x-axis labels by 45 degrees
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)

    # set the font size of the tick labels
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(16)
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(16)

    # title

    # add title + subtitle to plot
    plt.text(
        x=0.125,
        y=0.90,
        s=title,
        fontname=self.text_font_family,
        fontsize=24,
        ha="left",
        transform=fig.transFigure,
    )

    plt.text(
        x=0.125,
        y=0.86,
        s=subtitle,
        fontname=self.text_font_family,
        fontsize=18,
        ha="left",
        transform=fig.transFigure,
    )

    # line between titles and chart
    plt.gca().plot(
        [0.125, 0.9],  # x co-ords
        [0.80, 0.80],  # y co-ords
        transform=fig.transFigure,
        clip_on=False,
        color="k",
        linewidth=1.5,
    )

    # set the x and y labels
    ax.tick_params(axis="both", which="major", labelsize=16)
    plt.xlabel(xlabel, fontsize=20, fontname=self.text_font_family)
    plt.ylabel(ylabel, fontsize=20, fontname=self.text_font_family)

    # grid lines
    # keep only toned down vertical lines
    plt.grid(axis="y", alpha=0.4)
    plt.grid(axis="x", alpha=0.2)

    # turn off spines
    plt.gca().spines[["left", "right", "top"]].set_visible(False)

    # change space on top of chart we are actually adjusting the scale of the plot as well.
    plt.subplots_adjust(top=0.8, wspace=0.3)

    # return everything to the user
    return fig, ax

In [326]:
fig, ax = plt.subplots(figsize=(10, 6))
ylabel = "Title"
xlabel = "Duration"
title = "Top 10 Applications Used"
subtitle = f"Duration is the hours spent on the application from {data['Start Time'].min().strftime('%d %B %Y')} to {data['Start Time'].max().strftime('%d %B %Y')}"
text_font_family = "Product Sans"
code_font_family = "Jetbrains Mono"
plt.rcParams["font.family"] = code_font_family


# ax.bar(
#     x=collab_data[:10]["Title"],
#     height=collab_data[:10]["Real Duration"].dt.total_seconds() / 3600,
#     color="skyblue",
# )

# axes

sns.barplot(
    y=collab_data[:10]["Title"],
    x=collab_data[:10]["Real Duration"].dt.total_seconds() / 3600,
    ax=ax,
    palette="viridis",
    hue=collab_data[:10]["Title"],
)

# # tilt the x-axis labels by 45 degrees
# for tick in ax.get_xticklabels():
#     tick.set_rotation(90)

# set the font size of the tick labels
for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(13)
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(13)

# title

# add title + subtitle to plot
plt.text(
    x=0.125,
    y=0.90,
    s=title,
    fontname=text_font_family,
    fontsize=24,
    ha="left",
    transform=fig.transFigure,
)

plt.text(
    x=0.125,
    y=0.86,
    s=subtitle,
    fontname=text_font_family,
    fontsize=18,
    ha="left",
    transform=fig.transFigure,
)

# line between titles and chart
plt.gca().plot(
    [0.125, 0.9],  # x co-ords
    [0.80, 0.80],  # y co-ords
    transform=fig.transFigure,
    clip_on=False,
    color="k",
    linewidth=1.5,
)

# set the x and y labels
ax.tick_params(axis="both", which="major", labelsize=16)
plt.xlabel(xlabel, fontsize=20, fontname=text_font_family)
plt.ylabel(ylabel, fontsize=20, fontname=text_font_family)

# grid lines
# keep only toned down vertical lines
plt.grid(axis="y", alpha=0.4)
plt.grid(axis="x", alpha=0.2)

# turn off spines
plt.gca().spines[["left", "right", "top"]].set_visible(False)

# change space on top of chart we are actually adjusting the scale of the plot as well.
plt.subplots_adjust(top=0.8, wspace=0.3)
plt.show()

In [327]:
data.head()

Title          Start Time Registered End Time   Real Duration
1  idle 2024-02-05 18:14:10 2024-02-05 19:00:50 0 days 00:00:10
2  idle 2024-02-05 18:16:05 2024-02-05 18:56:19 0 days 00:00:10
3  idle 2024-02-05 18:17:22 2024-02-05 18:54:17 0 days 00:00:10
4  idle 2024-02-05 18:21:17 2024-02-05 18:54:07 0 days 00:00:10
5  idle 2024-02-05 18:28:57 2024-02-05 18:29:07 0 days 00:00:10

## lets find the app where I get most distracted 

In [365]:
# this would be an app where we are using it for more than a minute, and after less than 6 entires, it shows up again for more than a minute. This would be a good indicator of a break / distraction.

# lets collapse all the rows which are together.
# iterate over all the rows
new_data = []
for index, rows in data.iterrows():
    dict1 = {}

    # if the last row in new_data has the same title, add 10 seconds to the last row
    if len(new_data) != 0 and new_data[-1]["Title"] == rows["Title"]:
        new_data[-1].update(
            {
                "Real Duration": new_data[-1]["Real Duration"]
                + pd.Timedelta(seconds=10),
                "Registered End Time": rows["Registered End Time"],
            }
        )
    else:
        dict1["Title"] = rows["Title"]
        dict1["Start Time"] = rows["Start Time"]
        dict1["Registered End Time"] = rows["Registered End Time"]
        dict1["Real Duration"] = rows["Real Duration"]
        new_data.append(dict1)

# convert new_data to a dataframe
new_data = pd.DataFrame(new_data)
new_data.head(15)

Title          Start Time  \
0                                                idle 2024-02-05 18:14:10   
1                                             desktop 2024-02-05 18:54:07   
2                                            whatsapp 2024-02-05 18:54:17   
3                                             desktop 2024-02-05 18:54:23   
4                                        whatcha doin 2024-02-05 18:54:31   
5                                               games 2024-02-05 18:54:32   
6   krishnarajt/attendence-assistant: mini project... 2024-02-05 18:54:50   
7   editing attendence-assistant/readme.md at main... 2024-02-05 18:54:56   
8   krishnarajt/attendence-assistant: mini project... 2024-02-05 18:55:53   
9                                to-do | default view 2024-02-05 18:56:19   
10                                            desktop 2024-02-05 18:56:40   
11                                           whatsapp 2024-02-05 18:56:45   
12                               to-do | default view 2024-02-05 18:57:42   
13                                           whatsapp 2024-02-05 18:58:18   
14                                       whatcha doin 2024-02-05 18:58:46   

   Registered End Time   Real Duration  
0  2024-02-05 19:18:08 0 days 00:25:30  
1  2024-02-05 18:54:23 0 days 00:00:10  
2  2024-02-05 18:56:45 0 days 00:00:10  
3  2024-02-05 18:56:40 0 days 00:00:10  
4  2024-02-05 18:58:46 0 days 00:00:10  
5  2024-02-05 19:04:00 0 days 00:00:20  
6  2024-02-05 18:55:53 0 days 00:00:10  
7  2024-02-05 18:55:37 0 days 00:00:40  
8  2024-02-05 18:56:03 0 days 00:00:10  
9  2024-02-05 18:57:42 0 days 00:00:20  
10 2024-02-05 19:03:17 0 days 00:00:10  
11 2024-02-05 18:58:18 0 days 00:00:50  
12 2024-02-05 19:04:26 0 days 00:00:40  
13 2024-02-05 18:58:48 0 days 00:00:20  
14 2024-02-05 18:59:01 0 days 00:00:10

In [336]:
data

Title          Start Time Registered End Time   Real Duration
1               idle 2024-02-05 18:14:10 2024-02-05 19:00:50 0 days 00:00:10
154          desktop 2024-02-05 18:54:07 2024-02-05 18:54:23 0 days 00:00:10
155         whatsapp 2024-02-05 18:54:17 2024-02-05 18:56:45 0 days 00:00:10
156          desktop 2024-02-05 18:54:23 2024-02-05 18:56:40 0 days 00:00:10
157     whatcha doin 2024-02-05 18:54:31 2024-02-05 18:58:46 0 days 00:00:10
...              ...                 ...                 ...             ...
173496          idle 2024-04-10 14:56:21 2024-04-10 14:56:31 0 days 00:00:10
173497          idle 2024-04-10 14:56:31 2024-04-10 14:56:41 0 days 00:00:10
173498          idle 2024-04-10 14:56:41 2024-04-10 14:56:51 0 days 00:00:10
173499          idle 2024-04-10 14:56:51 2024-04-10 14:57:01 0 days 00:00:10
173500          idle 2024-04-10 14:57:01 2024-04-10 14:57:12 0 days 00:00:10

[152138 rows x 4 columns]